<a href="https://colab.research.google.com/github/liangchow/zindi-amazon-secret-runway/blob/explore-sentinel1-data/Data_Visualization/explore_sentinel1_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with github: [A guide from Google](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

### Imports and Setup.

In [1]:
%%capture
!pip -q install geojson
!pip -q install rasterio
!pip -q install eeconvert
!pip -q install geemap==0.17.3

In [2]:
# Standard imports
import os
import json

# Geospatial processing packages
import geopandas as gpd
import geojson
import shapely
import calendar
from shapely.geometry import box
from shapely.geometry import Polygon, Point
from pyproj import Transformer

# Mapping and plotting libraries
import ee
import eeconvert as eec
import geemap
import geemap.eefolium as emap
import folium

### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine. You will need to edit the following code cell to use your own account. All the Sentinel data will be downloaded through the Google Earth Engine.

In [3]:
ee.Authenticate()
ee.Initialize(project="ee-runway-detection")

Another way to authenticate is to create a IAM service account. Use the cell below if you want to go that route. Note that any images exported to Google Drive will be exported to Google Drive of the IAM service account and not your personal gmail account.

In [ ]:
# Load the service account key
# service_account = 'earth-engine-sa@ee-runway-detection.iam.gserviceaccount.com'
# credentials = ee.ServiceAccountCredentials(service_account, '/content/ee-runway-detection-ce5179ca616c.json')

# Authenticate with Earth Engine using the service account
# ee.Initialize(credentials)

### Mount your Google Drive and install project files

First, we'll mount your Google Drive. Then we'll clone the main branch from the GitHub repo so we have access to all of the files from there.

In [4]:
# mount your drive in case you have any new data uploaded there you want to use
from google.colab import drive
drive.mount('/content/drive')

# clone the main branch from GitHub to get all the data and files from there onto the current runtime session
!apt-get install git
!git clone https://github.com/liangchow/zindi-amazon-secret-runway.git
!git pull # pulls the latest changes from repo

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'zindi-amazon-secret-runway'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 165 (delta 58), reused 7 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (165/165), 792.45 KiB | 3.37 MiB/s, done.
Resolving deltas: 100% (58/58), done.
fatal: not a git repository (or any of the parent directories): .git


### Setup paths to the project files

In [5]:
airstrip_training_path = '/content/zindi-amazon-secret-runway/Data_Visualization/data/pac_2024_training/pac_2024_training.shp'
base_aoi_path = '/content/zindi-amazon-secret-runway/Data_Visualization//data/shp_test_AOIs'

# Select export folder on Google Drive
export_folder = 'Colab Notebooks'

In [6]:
# Read the shapefile for the training dataset
airstrip_training_gdf = gpd.read_file(airstrip_training_path)
airstrip_training_gdf.head(1)

,id,yr,largo,Activo,geometry
0,1,2023,968.918,0,"LINESTRING (-70.08929 -13.12984, -70.08053 -13..."


In [7]:
# Reproject the data to World Mercator projection
projected_airstrip_training_gdf = airstrip_training_gdf.to_crs(epsg=3395)
projected_airstrip_training_gdf.head(1)

,id,yr,largo,Activo,geometry
0,1,2023,968.918,0,"LINESTRING (-7802303.691 -1464870.283, -780132..."


### Download Sentinel1 Image Collection

Sentinel-1 collects C-band synthetic aperture radar (SAR) imagery at a variety of polarizations and resolutions. [Read more](https://developers.google.com/earth-engine/guides/sentinel1) about filtering the data to a homogenous susbet.



- Different polarization combinations are more and less sensitive to different kinds landcover features and phenomena. For example VV modes tend to pick up height/vertical features where VH modes tend to be more sensitive to surface textures. (Source)[https://support.capellaspace.com/hc/en-us/articles/360044738831-Sentinel-1-Polarization]

- For this application the most suitable product is the Interferometric Wide with dual polarisation. In fact, the cross-polarisation VH is more sensitive to change in vegetation density and structure while the polarisation ratio VH/VV is an important index for vegetation phenology or water content (European Wide Forest Classification Based on Sentinel-1 Data). (Source)[https://sentiwiki.copernicus.eu/web/s1-applications#S1Applications-ForestryS1-Applications-Forestry]

- Different polarization combinations provide different and complementary information about the surface features. For example, linearly oriented structures such as buildings or ripples in the sand tend to reflect and preserve the coherence (same linear direction) of the polarimetric signal. Randomly oriented structures such as tree leaves scatter and depolarize the signal as it bounces multiple times. (Source)[https://medium.com/@pulakeshpradhan/sentinel-1-sar-data-processing-2dd9b2751d95]

In [8]:
# Helper function to calculate VV/VH ratio and add it as a new band
def add_vv_vh_ratio(image):
    ratio_band = image.select('VV').divide(image.select('VH')).rename('VV_VH_Ratio')
    return image.addBands(ratio_band)

In [9]:
# Helper function to rename band
def rename_bands(image, suffix):
    # Get the original band names
    original_band_names = image.bandNames()

    # Create new band names using server-side expression
    new_band_names = original_band_names.map(lambda band: ee.String(band).cat('_').cat(suffix))

    # Rename all bands
    renamed_image_all = image.rename(new_band_names)

    return renamed_image_all

In [10]:
# Function to convert bands to Float32 to reduce file size
def to_float(image):
    return image.toFloat()

In [42]:
# @title
def download_sentinel1_monthly_data(aoi, id, year=2023):

  # Load Sentinel-1 data for ascending and descending orbits
  sentinel1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))

  print(f'Number of images in the collection: {sentinel1_collection.size().getInfo()}')

  # Process and export monthly composites
  for month in range(1, 13):
      # Define start and end dates for the month
      start_date = ee.Date.fromYMD(year, month, 1)
      end_date = ee.Date.fromYMD(year, month, calendar.monthrange(year, month)[1])

      # Filter Sentinel-1 data for the current month
      monthly_collection = sentinel1_collection.filterDate(start_date, end_date) \
                                        .map(lambda img: add_vv_vh_ratio(img)) \
                                        .map(lambda img: to_float(img))
      print(f'Number of images in {calendar.month_name[month]}: {monthly_collection.size().getInfo()}')

      # Calculate median composite for the current month
      monthly_median_composite = monthly_collection.median().clip(aoi)

      # # Define the export description and file name prefix
      export_description = f'S1_Monthly_Median_Composite_{id}_{year}_{month:02d}'
      file_name_prefix = f'S1_Monthly_Median_Composite_{id}_{year}_{month:02d}'

      # # # Export the combined composite to Google Drive
      task = ee.batch.Export.image.toDrive(
          image=monthly_median_composite.select(['VV', 'VH', 'VV_VH_Ratio']),
          description=export_description,
          folder='GEE_Exports',
          fileNamePrefix=file_name_prefix,
          crs=proj_3395,
          scale=10,
          region=aoi,
          maxPixels=1e13,
          fileFormat='GeoTIFF'
      )

      # # Start the export task
      task.start()

      print(f'Export task for {calendar.month_name[month]} started. Check your Google Drive for the exported image.')

In [39]:
def download_sentinel1_annual_data(aoi, id, year=2023):

  # Load Sentinel-1 data for ascending and descending orbits
  sentinel1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))

  print(f'Number of images in the collection: {sentinel1_collection.size().getInfo()}')

  start_year = year - 1
  end_year = year + 1

  # Process and export monthly composites
  for year in range(start_year, end_year):
      # Define start and end dates for the year
      start_date = ee.Date.fromYMD(year, 1, 1)
      end_date = ee.Date.fromYMD(year, 12, 31)

      # Filter Sentinel-1 data for the current year
      annual_collection = sentinel1_collection.filterDate(start_date, end_date) \
                                        .map(lambda img: add_vv_vh_ratio(img)) \
                                        .map(lambda img: to_float(img))

      # Calculate median composite for the current year
      annual_median_composite = annual_collection.median().clip(aoi)

      # # Define the export description and file name prefix
      export_description = f'S1_Annual_Median_Composite_{id}_{year}'
      file_name_prefix = f'S1_Annual_Median_Composite_{id}_{year}'

      # # # Export the combined composite to Google Drive
      task = ee.batch.Export.image.toDrive(
          image=annual_median_composite.select(['VV', 'VH', 'VV_VH_Ratio']),
          description=export_description,
          folder='GEE_Exports',
          fileNamePrefix=file_name_prefix,
          crs=proj_3395,
          scale=10,
          region=aoi,
          maxPixels=1e13,
          fileFormat='GeoTIFF'
      )

      # # Start the export task
      task.start()

      print(f'Export task for {year} started. Check your Google Drive for the exported image.')

In [37]:
# EPSG:3395 (World Mercator) as Proj4 string
proj_3395 = 'PROJCS["WGS 84 / World Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],AUTHORITY["EPSG","3395"],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

### Process  single runway

In [36]:
# Helper function to convert geopandas feature geometry to Earth Engine geometry for plotting
def gpd_geometry_to_ee_geometry(gpd_geometry, target_crs='EPSG:3395'):
  # Convert the reprojected GeoPandas geometry to GeoJSON-like format
  geojson_geometry = gpd_geometry.__geo_interface__

  # Define the CRS
  crs = ee.Projection(target_crs)

  # Convert the GeoJSON geometry to Earth Engine geometry
  ee_geometry = ee.Geometry(geojson_geometry, proj=crs)

  return ee_geometry

In [40]:
# Select airstrip record
airstrip_gpd = projected_airstrip_training_gdf.iloc[22]

# Get information from the airstrip
year = int(airstrip_gpd.yr)
id = airstrip_gpd.id
airstrip_gpd_geometry = airstrip_gpd.geometry

# Create a square buffer at a distance of 1 km around centroid of airstrip
buffer_gpd = Point(airstrip_gpd_geometry.centroid).buffer(1000, cap_style=3)

# Convert GeoPandas geometry to Earth Engine features
airstrip_ee = gpd_geometry_to_ee_geometry(airstrip_gpd_geometry)
buffer_ee = gpd_geometry_to_ee_geometry(buffer_gpd)

# Download monthly composites
# GEE expects the aoi to be in geogprahic coordinates to filter the image collection

#img = download_sentinel1_data(buffer_ee, id)
#download_sentinel1_monthly_data(buffer_ee, id, year)
download_sentinel1_annual_data(buffer_ee, id, year)

Number of images in the collection: 420
Export task for 2021 started. Check your Google Drive for the exported image.
Export task for 2022 started. Check your Google Drive for the exported image.


### Process single Test AOI

In [30]:
# Load the shape file and import into geopanda dataframe
test_gdf = gpd.read_file(f'{base_aoi_path}/aoi_2020_01.shp')
test_gdf.head()

,MINX,MINY,MAXX,MAXY,CNTX,CNTY,AREA,PERIM,HEIGHT,WIDTH,geometry
0,690096.799317,8.793048e+06,705506.799317,8.808318e+06,697801.799317,8.800683e+06,235310700.0,61360.0,15270.0,15410.0,"POLYGON ((690096.799 8793048.011, 690096.799 8..."


In [31]:
# Reproject to EPSG:3395
test_gdf_3395 = test_gdf.to_crs(epsg=3395)
test_gdf_3395.head()

,MINX,MINY,MAXX,MAXY,CNTX,CNTY,AREA,PERIM,HEIGHT,WIDTH,geometry
0,690096.799317,8.793048e+06,705506.799317,8.808318e+06,697801.799317,8.800683e+06,235310700.0,61360.0,15270.0,15410.0,"POLYGON ((-8155337.243 -1214204.7, -8155426.03..."


In [41]:
# Select airstrip record
test_gpd = test_gdf_3395.iloc[0]

# Get information from the airstrip
year = 2023
id = 'Test'
test_gpd_geometry = test_gpd.geometry

# Convert GeoPandas geometry to Earth Engine features
test_ee = gpd_geometry_to_ee_geometry(test_gpd_geometry)

# Download annual composite for test area
download_sentinel1_annual_data(test_ee, id, year)

Number of images in the collection: 420
Export task for 2022 started. Check your Google Drive for the exported image.
Export task for 2023 started. Check your Google Drive for the exported image.


###Plot the images.
Modify the download scripts to return the image composites instead of exporting to geotiff files. Or download images from export folder in Google drive to plot on map.

In [ ]:
# @title
# Create a map
m = geemap.Map()

# Visualization params
vis_params = {
    'min': -30,
    'max': 100,
    #'palette': ['blue', 'green', 'red']
}

# Display the map
m.centerObject(buffer_ee, zoom=15)

# Enable layer control
m.addLayerControl()

# Add layers
m.addLayer(img[1], vis_params, 'SAR')
m.addLayer(airstrip_ee, {'color': 'red', 'width': 1}, 'Airstrip')

m

Map(center=[-13.616470429757568, -69.17015088567591], controls=(WidgetControl(options=['position', 'transparen…

### Loop over all runways

In [ ]:
# Function to download annual composites for one airstrip
def export_images_by_airstrip(airstrip_gpd):
  # Get information from the airstrip
  year = int(airstrip_gpd.yr)
  id = airstrip_gpd.id
  airstrip_gpd_geometry = airstrip_gpd.geometry

  # Create a square buffer at a distance of 5 km around centroid of airstrip
  buffer_gpd = Point(airstrip_gpd_geometry.centroid).buffer(1000, cap_style=3)

  # Convert GeoPandas geometry to Earth Engine features
  airstrip_ee = gpd_geometry_to_ee_geometry(airstrip_gpd_geometry)
  buffer_ee = gpd_geometry_to_ee_geometry(buffer_gpd)

  # Download monthly composites
  # GEE expects the aoi to be in geogprahic coordinates to filter the image collection

  #img = download_sentinel1_data(buffer_ee, id)
  #download_sentinel1_monthly_data(buffer_ee, id, year)
  download_sentinel1_annual_data(buffer_ee, id, year)
  return None

In [ ]:
# Iterate through all airstrips
#for index, row in projected_airstrip_training_gdf.iterrows():
  #export_images_by_airstrip(row)

###Extra
If you are using a service account, the following workflow can be used to download images from Google Drive associated with service account. This code is a WIP.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

# authenticate to Google Drive (of the Service account)
#service_account = 'earth-engine-sa@ee-runway-detection.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, '/content/ee-runway-detection-ce5179ca616c.json')

gauth = GoogleAuth()
scopes = ['https://www.googleapis.com/auth/drive']
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/ee-runway-detection-ce5179ca616c.json', scopes=scopes)

drive = GoogleDrive(gauth)

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload

# Path to your service account key file
SERVICE_ACCOUNT_FILE = '/content/ee-runway-detection-ce5179ca616c.json'

# Define the required scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize the Google Drive service
drive_service = build('drive', 'v3', credentials=credentials)

# Function to download a file from Google Drive
def download_file(file_id, file_name):
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(file_name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}% complete.")
    print(f"File {file_name} downloaded successfully.")

# Example: List files in the service account's Google Drive
results = drive_service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

# Print the list of files in the Drive
if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(f"{item['name']} ({item['id']})")

# Download a specific file by its file ID
# Replace 'your-file-id' with the actual file ID and 'your-file-name' with the desired name
file_id = '1laCSccscSYJcC2RmiOCRHyfB8qXlc5WH'  # Replace with the file ID you want to download
file_name = 'Sentinel1_Composite_2_2023_01.tif'  # Replace with the desired local file name
download_file(file_id, file_name)


Files:
Sentinel1_Composite_2_2023_01.tif (1laCSccscSYJcC2RmiOCRHyfB8qXlc5WH)
Sentinel1_Composite_2_2023_02.tif (10DnqxGv1dSktCLB40Syb5tmR0weggm3p)
Colab Notebooks/GEE_Exports (1sJip4DGO9ABQ8DM28ESt-5wW6iIYjU5C)
Colab Notebooks/GEE_Exports (17ujqiU1kbte753XzgtH3VtCl5SS_SSwe)
GEE_Exports (1b1ZqdSvjijkgyX3t9qUw1bZyqcbdJ9w_)
Sentinel1_Composite_2_2023_02.tif (14-0RMjA_vMAUWs-ThTQcsKr18ke0IImV)
Sentinel1_Composite_2_2023_01.tif (1WsJlc9g37S2WiIGCF5UXysnggFPS9P4s)
Sentinel1_Composite_2_2023_01.tif (17E8BUf2o9nyguuIjhUPJetq5XWMYJWxs)
Sentinel1_Composite_2_2023_02.tif (1YIkQ1H5tiiSE4eCnvJmGWrnp0IPSUXJ5)
Colab Notebooks (1J5U7Ofd0z5gmKzof_GNvQh2pT1YSY6kl)
Download 100% complete.
File Sentinel1_Composite_2_2023_01.tif downloaded successfully.
